<a href="https://colab.research.google.com/github/matiasmazzucchi/Bicycle-scraper/blob/main/Bicycle_scraper(28/5/24).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CODIGO PARA EXTRAER KROMOS, BERTOLLINA Y FACHIN BIKES.
---

In [ ]:
#llamar a las siguientes bibliotecas
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import csv
import datetime
import gspread


#URL DE CADA PAGINA (terminados en 'K'es de la pagina Kromos y terminados en 'B' es de la pagina de bertollina, terminado con 'F' es de la pagina de Fachin)
URL_BASE_K = 'https://www.kromosbicicletas.com.ar/productos/mar/1/venzo'
URL_BASE_B = 'https://www.bertolinabikes.com/search/?q=venzo&mpage=2'
URL_BASEF = 'https://www.facchinbicycles.com.ar/listado/deportes-fitness/ciclismo/bicicletas/venzo/'

#Obtener el HTML de la página
pedido_obtenidoK = requests.get(URL_BASE_K)
html_obtenidoK = pedido_obtenidoK.text
pedido_obtenidoB = requests.get(URL_BASE_B)
html_obtenidoB = pedido_obtenidoB.text
pedido_obtenidoF = requests.get(URL_BASEF)
html_obtenidoF = pedido_obtenidoF.text

#Parsear el HTML
soupK = BeautifulSoup(html_obtenidoK, 'html.parser')
soupB = BeautifulSoup(html_obtenidoB, 'html.parser')
soupF = BeautifulSoup(html_obtenidoF, 'html.parser')



# Encontrar todos los títulos y precios
#KROMOS
bicicletasK = soupK.find_all('h2', class_='tt-title')
precioK = soupK.find_all('div', class_="tt-price")
#BERTOLLINA
bicicletasB = soupB.find_all('a', class_='js-item-name')
preciosB = soupB.find_all('span', class_='js-price-display item-price h6')
precios_viejosB = soupB.find_all('span', class_="js-compare-price-display item-price-compare p-none-left-xs p-right-quarter-xs")
#FACHIN
bicicletaF = soupF.find_all('h2', class_='ui-search-item__title shops__item-title')
preciosF = soupF.find_all('div', class_='ui-search-price ui-search-price--size-medium shops__price')


#Creating the lists for the next dataframe
#KROMOS
bicicletasKL = []
precioKL = []
#BERTOLLINA
bicicletaBL = []
preciosBL = []
precios_viejosBL = []
#FACHIN
bicicletaLF = []
preciosLF = []




#Ciclos for para limpiar los datos y obtener los nombres y bicicletas
#KROMOS
for bicicleta in bicicletasK:
 bicicletasKL.append(bicicleta.get_text(strip=True))

for precios in precioK:
  precioKL.append(precios.get_text(strip=True))

#BERTOLLINA
for bicicletas in bicicletasB:
  bicicletaBL.append(bicicletas.get_text(strip=True))

for precios in preciosB:
  preciosBL.append(precios.get_text(strip=True))


for precio_viejo in precios_viejosB:
  precios_viejosBL.append(precio_viejo.get_text(strip=True))

#FACHIN
for bicicletas in bicicletaF:
  bicicletaLF.append(bicicletas.get_text(strip=True))

for precio in preciosF:
  preciosLF.append(precio.get_text(strip=True))




#Armado del dataframe
#KROMOS DATAFRAME
ListaFinalK = list(zip(bicicletasKL, precioKL))
dfK = pd.DataFrame(ListaFinalK, columns=['Bicicleta','Precio'])

#BERTOLLINA DATAFRAME
ListaFinalB = list(zip(bicicletaBL, preciosBL, precios_viejosBL))
dfB = pd.DataFrame(ListaFinalB, columns=['Bicicleta','Precio', 'Precios Viejos'])

#FACHIN DATAFRAME
ListaFinalF = list(zip(bicicletaLF, preciosLF))
dfF = pd.DataFrame(ListaFinalF, columns=['Bicicleta','Precio'])


#Limpieza de los Data Frame

dfB['Precios Viejos'] = dfB['Precios Viejos'].str.replace('.', '')
dfB['Precio'] = dfB['Precio'].str.replace('.', '')
dfK['Precio'] = dfK['Precio'].str.replace('.', '')
dfK['Precio'] = dfK['Precio'].str[:-2]
dfF['Precio'] = dfF['Precio'].apply(lambda x: x[8:] if 'OFF' in x else x)
dfF['Precio'] = dfF['Precio'].str.replace('.', '')
dfF['Precio'] = dfF['Precio'].str.replace('%', '')
dfF['Precio'] = dfF['Precio'].str.replace('OFF', '')


# Añadir las columnads de fechas con las fechas del dia de la extraccion de los datos
now = datetime.datetime.now()
formatted_date = now.strftime("%Y-%m-%d")
dfK["Fecha de extraccion"] = formatted_date
dfB["Fecha de extraccion"] = formatted_date
dfF["Fecha de extraccion"] = formatted_date

# Agregar la columna de la empresa de la extraccion con la impresion del nombre de la marca

dfB['Empresa'] = 'Bertolina'
dfK['Empresa'] = 'Kromos'
dfF['Empresa'] = 'Fachin'

# Dejar en minuscula a todos los registros para su posterior analisis y comparasion

dfK['Bicicleta'] = dfK['Bicicleta'].str.lower()
dfB['Bicicleta'] = dfB['Bicicleta'].str.lower()
dfF['Bicicleta'] = dfF['Bicicleta'].str.lower()






In [ ]:
dfB


,Bicicleta,Precio,Precios Viejos,Fecha de extraccion,Empresa
0,bicicleta venzo x-blaze rodado 29,$2454880,$2785805,2024-05-13,Bertolina
1,bicicleta venzo primal rodado 29 freno h...,$521138,$694850,2024-05-13,Bertolina
2,bicicleta rodado 29 venzo loki shadow 21...,$439200,$481999,2024-05-13,Bertolina
3,bicicleta venzo atix ex rodado 29 - shim...,$1444000,$0,2024-05-13,Bertolina
4,bicicleta de ruta venzo phoenix rodado 2...,$963700,$1043200,2024-05-13,Bertolina
5,bicicleta de ruta venzo phoenix rodado 2...,$802597,$940316,2024-05-13,Bertolina
6,venzo raptor rodado 29 2x9 shimano freno...,$699300,$991229,2024-05-13,Bertolina
7,bicicleta venzo atix ex 2x11 sram,$1839856,$1948726,2024-05-13,Bertolina
8,bicicleta rodado 29 venzo skyline evo 21...,$524300,$674845,2024-05-13,Bertolina
9,bicicleta rodado 29 venzo skyline shadow...,$524300,$674845,2024-05-13,Bertolina


In [ ]:

# Autenticacion y permisos de uso de la API de google SHEETS
from google.colab import auth
auth.authenticate_user()

from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


In [ ]:
##--ESTA CELDA SOLO SE EJECUTA CUANDO LA HOJA NO ESTA CREADA, SI YA ESTA CREADA EN DRIVE SOLO SE DEBE EJECUTAR EL AUTENTICADOR--##

#CREACION de la hoja de caluclo de google sheets
sh = gc.create('Bicicletas Web scraping Gspread KROMOS, BERTOLLINA AND FACHIN')

#Crado de las hojas dentro de  'Bicicletas Web scraping Gspread KROMOS AND BERTOLLINA'

worksheet1 = gc.open('Bicicletas Web scraping Gspread KROMOS, BERTOLLINA AND FACHIN').add_worksheet(title="Kromos", rows=100, cols=20)
worksheet2 = gc.open('Bicicletas Web scraping Gspread KROMOS, BERTOLLINA AND FACHIN').add_worksheet(title="Bertollina", rows=100, cols=20)
worksheet3 = gc.open('Bicicletas Web scraping Gspread KROMOS, BERTOLLINA AND FACHIN').add_worksheet(title="Fachin", rows=100, cols=20)

# #------------------------------------------------------------------------------------------------------------------------------------##########
# %%

In [ ]:

#Agregado del Open de las distintas hojas
worksheetK = gc.open('Bicicletas Web scraping Gspread KROMOS, BERTOLLINA AND FACHIN').worksheet('Kromos')
worksheetB = gc.open('Bicicletas Web scraping Gspread KROMOS, BERTOLLINA AND FACHIN').worksheet('Bertollina')
worksheetF = gc.open('Bicicletas Web scraping Gspread KROMOS, BERTOLLINA AND FACHIN').worksheet('Fachin')




In [ ]:


#Apertura de todas las hojas y almacenamiento de los datos en las hojas correspondientes
worksheetK.update([dfK.columns.values.tolist()] + dfK.values.tolist())
worksheetB.update([dfB.columns.values.tolist()] + dfB.values.tolist())
worksheetF.update([dfF.columns.values.tolist()] + dfF.values.tolist())

{'spreadsheetId': '1pFm9x2Hd5ZKPVD49CDyLTfAbSHUKER0kGTZdVS34a_Q',
 'updatedRange': 'Fachin!A1:D51',
 'updatedRows': 51,
 'updatedColumns': 4,
 'updatedCells': 204}



---



In [ ]:
##REALIZAR NUEVAS EXTRAXIONES

Una vez creada la hoja de calculo y habiendo alimentado las hojas de google sheets con los datos lo siguientes acondicionar los datos ya en Googl Sheets.

Para esto se debe:
*Crear una macro que realise los siguentes procesos.
- Alimentar la hoja 1 creada automaticamente, con los datos de las 3 hojas creadas. (Sino tambien se puede obtar por convinar las tablas)
- Combertir todos los datos $ en numeros regulares.
- Cambiar el Formato de las fechas en Google Sheets


In [ ]:
#ESTE ES EL CODIGO COMPLETO PARA DESCARGAR DIRECTAMENTE EL CSV Y HACERLO CON LOS METODOS VIEJOS
#bicicletasL.insert(0, "Bicicletas")
#precioL.insert(0, "Precio")
#datos = dict(zip(bicicletasL, precioL))
#with open ("datos.csv", "w") as f:
 # w = csv.writer(f)
 # w.writerows(datos.items())



---

